In [357]:
import numpy as np
import pandas as pd
import random
import matplotlib.pyplot as plt
import seaborn as sns
from Bio import SeqIO
from Bio.Seq import Seq
from Bio.Alphabet import IUPAC
from Bio.SeqRecord import SeqRecord
from Bio import SeqUtils
from sklearn.model_selection import cross_val_score

In [2]:
df = pd.read_csv('final file hopefully no errors.csv')
df['Len'] = [len(s) for s in df['Seq'].to_list()]

dfp = df.loc[df['Gen'] == 'ssRNA(+)'].reset_index(drop = True)
dfn = df.loc[df['Gen'] == 'ssRNA(-)'].reset_index(drop = True)

res = pd.read_csv('res 340.csv')
resp = res.iloc[:2862, :].reset_index(drop = True)
resn = res.iloc[2862:, :].reset_index(drop = True)

In [16]:
rpd = pd.concat([resp.quantile(q=0.05), resp.quantile(q=0.10), resp.quantile(q=0.90), resp.quantile(q=0.95)], axis=1)
rnd = pd.concat([resn.quantile(q=0.05), resn.quantile(q=0.10), resn.quantile(q=0.90), resn.quantile(q=0.95)], axis=1)

In [30]:
odf = pd.DataFrame(index = rpd.index, columns= rpd.columns)

for i in rpd.index:
    odf.loc[i,0.05] = max(rpd.loc[i,0.05], rnd.loc[i,0.05])
    odf.loc[i,0.10] = max(rpd.loc[i,0.10], rnd.loc[i,0.10])
    odf.loc[i,0.90] = min(rpd.loc[i,0.90], rnd.loc[i,0.90])
    odf.loc[i,0.95] = min(rpd.loc[i,0.95], rnd.loc[i,0.95])

In [35]:
odf['in 90'] = 0
odf['in 80'] = 0

for i in res.index:
    for c in res.columns:
        if odf.loc[c,0.05] < res.loc[i,c] < odf.loc[c,0.95]:
            odf.loc[c, 'in 90'] = odf.loc[c, 'in 90'] + 1
            
            if odf.loc[c,0.10] < res.loc[i,c] < odf.loc[c,0.90]:
                odf.loc[c, 'in 80'] = odf.loc[c, 'in 80'] + 1

In [60]:
odf.sort_values(by=['in 80', 'in 90'])[:5]

,0.05,0.1,0.9,0.95,in 90,in 80
ATC,0.0341546,0.0372251,0.0341369,0.0363116,150,0
ATCA,0.011796,0.0126798,0.0121456,0.0131263,242,0
ATCT,0.00917614,0.00995548,0.00977367,0.010607,350,0
TATC,0.00725136,0.00851144,0.0085953,0.00958841,845,16
TGGT,0.00692177,0.00852914,0.0088329,0.00927491,508,57


In [ ]:
Challenger

In [358]:
def split(df1):
    p = df1.loc[:2861].sample(frac=1).reset_index(drop = True)
    n = df1.loc[2862:].sample(frac=1).reset_index(drop = True)
    ps1 = p.iloc[:1144]
    ns1 = n.iloc[:112]
    ps2 = p.iloc[1144:2288].reset_index(drop = True)
    ns2 = n.iloc[112:224].reset_index(drop = True)
    ps3 = p.iloc[2288:].reset_index(drop = True)
    ns3 = n.iloc[224:].reset_index(drop = True)
    
    return (ps1,ns1,ps2,ns2,ps3,ns3)

In [359]:
def featsrc(rp1, rn1):    
    
    res = pd.concat([rp1,rn1], axis = 0).reset_index(drop = True)
    
    rpd = pd.concat([rp1.quantile(q=0.05), rp1.quantile(q=0.10), rp1.quantile(q=0.90), rp1.quantile(q=0.95)], axis=1)
    rnd = pd.concat([rn1.quantile(q=0.05), rn1.quantile(q=0.10), rn1.quantile(q=0.90), rn1.quantile(q=0.95)], axis=1)
    
    odf = pd.DataFrame(index = rpd.index, columns= rpd.columns)

    for i3 in rpd.index:
        odf.loc[i3,0.05] = max(rpd.loc[i3,0.05], rnd.loc[i3,0.05])
        odf.loc[i3,0.10] = max(rpd.loc[i3,0.10], rnd.loc[i3,0.10])
        odf.loc[i3,0.90] = min(rpd.loc[i3,0.90], rnd.loc[i3,0.90])
        odf.loc[i3,0.95] = min(rpd.loc[i3,0.95], rnd.loc[i3,0.95])
    
    odf['in 90'] = 0
    odf['in 80'] = 0

    for i in res.index:
        for c in res.columns:
            if odf.loc[c,0.05] < res.loc[i,c] < odf.loc[c,0.95]:
                odf.loc[c, 'in 90'] = odf.loc[c, 'in 90'] + 1

                if odf.loc[c,0.10] < res.loc[i,c] < odf.loc[c,0.90]:
                    odf.loc[c, 'in 80'] = odf.loc[c, 'in 80'] + 1
    
    print(odf.sort_values(by=['in 80', 'in 90'])[:5])
    return odf.sort_values(by=['in 80', 'in 90'])[:7].index

In [374]:
def rfimp(feats,pnrtr,pnrts):
    
    r1 = pnrtr[feats].copy()
    r2 = pnrts[feats].copy()
    pnrtr = pnrtr.iloc[:,:8].drop(columns=['Id', 'MolTyp', 'Topol', 'Des', 'Len'])
    pnrts = pnrts.iloc[:,:8].drop(columns=['Id', 'MolTyp', 'Topol', 'Des', 'Len'])
    
    pnrtr = pd.concat([pnrtr, r1], axis = 1)
    pnrts = pd.concat([pnrts, r2], axis = 1)
         
    #Splitting features and labels
    X_train = pnrtr.iloc[:, 3:].values.astype(float)
    X_test = pnrts.iloc[:, 3:].values.astype(float)
    y_train = pnrtr.iloc[:, 1].values.astype(float)
    y_test = pnrts.iloc[:, 1].values.astype(float)
    
    X = np.concatenate([X_train, X_test], axis = 0)
    y = np.concatenate([y_train, y_test], axis = 0)
       
    # Feature Scaling
    from sklearn.preprocessing import StandardScaler
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    # Fitting Random Forest Classification to the Training set
    from sklearn.ensemble import RandomForestClassifier
    classifier = RandomForestClassifier(n_estimators = 15, criterion = 'entropy', random_state = 0)
    classifier.fit(X_train, y_train)
    
    # Predicting the Test set results
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    from sklearn.metrics import confusion_matrix
    cm = confusion_matrix(y_test, y_pred)
    
    scores_mic = cross_val_score(classifier, X, y, cv=5, scoring='f1_micro')
    
    return scores_mic

In [361]:
pndf = pd.read_csv('pndf.csv')
res = pd.read_csv('res 340.csv')
df1 = pd.concat([pndf, res], axis = 1)
df1.loc[pndf['Gen'] == 'ssRNA(+)', 'Gen'] = 0
df1.loc[pndf['Gen'] == 'ssRNA(-)', 'Gen'] = 1

aclist = []

for i in range(20):
    a,b,c,d,e,f = split(df1)
    ps1 = a.iloc[:,:8]
    ns1 = b.iloc[:,:8]
    rp1 = a.iloc[:,8:]
    rn1 = b.iloc[:,8:]
    pnrtr = pd.concat([c,d], axis = 0)
    pnrts = pd.concat([e,f], axis = 0)    
    
    feats = featsrc(rp1,rn1)
    fmic = rfimp(feats, pnrtr, pnrts)

    aclist.append(fmic.mean())
    print(fmic.mean(), fmic.std()*2)

print(np.mean(aclist))

            0.05         0.1         0.9        0.95  in 90  in 80
ATC    0.0355576   0.0380317   0.0339401   0.0363014     15      0
ATCA   0.0121301    0.012758   0.0122555   0.0132284     81      0
ATCT  0.00932914  0.00995803  0.00987226    0.010615    138      0
TGGT  0.00688647  0.00853172   0.0088829  0.00933357    224     16
GTG    0.0246069   0.0279954   0.0283972   0.0298423    420     16


NameError: name 'X' is not defined

Negative control

In [378]:
pndf = pd.read_csv('pndf.csv')
res = pd.read_csv('res 340.csv')
df1 = pd.concat([pndf, res], axis = 1)

for i1 in range(10):
    df1['Gen'] = 0
    rindex = np.random.choice(len(df1), 283, replace=False)
    for i2 in rindex:
        df1.loc[i2,'Gen'] = 1

    aclist = []

    for i2 in range(5):
        a,b,c,d,e,f = split(df1)
        ps1 = a.iloc[:,:8]
        ns1 = b.iloc[:,:8]
        rp1 = a.iloc[:,8:]
        rn1 = b.iloc[:,8:]
        pnrtr = pd.concat([c,d], axis = 0)
        pnrts = pd.concat([e,f], axis = 0)    

        feats = featsrc(rp1,rn1)
        fmic = rfimp(feats, pnrtr, pnrts)

        aclist.append(fmic.mean())
        print(fmic.mean())
        print(fmic.mean(), fmic.std())
            
    print(np.mean(aclist))

            0.05         0.1         0.9        0.95  in 90  in 80
ATC    0.0347601   0.0374557   0.0340281   0.0365468     40      0
ATCA   0.0115689   0.0124057   0.0120387   0.0130489    125      0
ATCT  0.00918498  0.00995653  0.00975332   0.0106885    132      0
TGGT  0.00678443  0.00818278  0.00859503  0.00943402    275     31
TATC  0.00651174  0.00825113  0.00855566  0.00964052    480     36
[0.90740741 0.91005291 0.91005291 0.9047619  0.90981432]
0.9084178911765118 0.002082494230382567
            0.05         0.1         0.9        0.95  in 90  in 80
ATCA   0.0114845   0.0125011   0.0122382   0.0132479    137      0
ATC    0.0323999   0.0348864   0.0341313    0.036645    160      0
TGGT  0.00704999  0.00870353  0.00893852  0.00927366    181     16
ATCT  0.00865733    0.009684  0.00990213    0.010639    251     19
TATC  0.00657106  0.00821622  0.00856492  0.00951696    484     54
[0.91269841 0.91269841 0.9047619  0.9021164  0.90185676]
0.9068263792401723 0.004901113539202913
  

KeyboardInterrupt: 

In [ ]:
#advanced feat finder

In [382]:
def featsrc(rp1, rn1):    
    
    res = pd.concat([rp1,rn1], axis = 0).reset_index(drop = True)
    
    rpd = pd.concat([rp1.quantile(q=0.05), rp1.quantile(q=0.10), rp1.quantile(q=0.90), rp1.quantile(q=0.95)], axis=1)
    rnd = pd.concat([rn1.quantile(q=0.05), rn1.quantile(q=0.10), rn1.quantile(q=0.90), rn1.quantile(q=0.95)], axis=1)
    
    odf = pd.DataFrame(index = rpd.index, columns= rpd.columns)

    for i3 in rpd.index:
        odf.loc[i3,0.05] = max(rpd.loc[i3,0.05], rnd.loc[i3,0.05])
        odf.loc[i3,0.10] = max(rpd.loc[i3,0.10], rnd.loc[i3,0.10])
        odf.loc[i3,0.90] = min(rpd.loc[i3,0.90], rnd.loc[i3,0.90])
        odf.loc[i3,0.95] = min(rpd.loc[i3,0.95], rnd.loc[i3,0.95])
    
    odf['in 90'] = 0
    odf['in 80'] = 0

    for i in res.index:
        for c in res.columns:
            if odf.loc[c,0.05] < res.loc[i,c] < odf.loc[c,0.95]:
                odf.loc[c, 'in 90'] = odf.loc[c, 'in 90'] + 1

                if odf.loc[c,0.10] < res.loc[i,c] < odf.loc[c,0.90]:
                    odf.loc[c, 'in 80'] = odf.loc[c, 'in 80'] + 1
    
    return odf.sort_values(by=['in 80', 'in 90'])[:200].index

#challenger
pndf = pd.read_csv('pndf.csv')
res = pd.read_csv('res 340.csv')
df1 = pd.concat([pndf, res], axis = 1)
df1.loc[pndf['Gen'] == 'ssRNA(+)', 'Gen'] = 0
df1.loc[pndf['Gen'] == 'ssRNA(-)', 'Gen'] = 1

fdf = pd.DataFrame(index = res.columns, columns = ['challenger', 'control'])
fdf['challenger'] = 0
fdf['control'] = 0

for i in range(50):
    a,b,c,d,e,f = split(df1)
    ps1 = a.iloc[:,:8]
    ns1 = b.iloc[:,:8]
    rp1 = a.iloc[:,8:]
    rn1 = b.iloc[:,8:]
    
    feats = featsrc(rp1,rn1)
    
    c = 200
    for f in feats:
        fdf.loc[f,'challenger'] = fdf.loc[f,'challenger'] + c
        c-=1
        
    print('%d challenger iterations remaining.' % (50-i))

    
#control
for i in range(25):
    df1['Gen'] = 0
    rindex = np.random.choice(len(df1), 283, replace=False)
    for i2 in rindex:
        df1.loc[i2,'Gen'] = 1
        
    for i2 in range(2):
        a,b,c,d,e,f = split(df1)
        ps1 = a.iloc[:,:8]
        ns1 = b.iloc[:,:8]
        rp1 = a.iloc[:,8:]
        rn1 = b.iloc[:,8:]

        feats = featsrc(rp1,rn1)

        c = 200
        for f in feats:
            fdf.loc[f,'control'] = fdf.loc[f,'control'] + c
            c-=1
    print('%d control iterations remaining.' % (25 - i))
            
fdf['result'] = fdf['challenger'] - fdf['control']

50 challenger iterations remaining.
49 challenger iterations remaining.
48 challenger iterations remaining.
47 challenger iterations remaining.
46 challenger iterations remaining.
45 challenger iterations remaining.
44 challenger iterations remaining.
43 challenger iterations remaining.
42 challenger iterations remaining.
41 challenger iterations remaining.
40 challenger iterations remaining.
39 challenger iterations remaining.
38 challenger iterations remaining.
37 challenger iterations remaining.
36 challenger iterations remaining.
35 challenger iterations remaining.
34 challenger iterations remaining.
33 challenger iterations remaining.
32 challenger iterations remaining.
31 challenger iterations remaining.
30 challenger iterations remaining.
29 challenger iterations remaining.
28 challenger iterations remaining.
27 challenger iterations remaining.
26 challenger iterations remaining.
25 challenger iterations remaining.
24 challenger iterations remaining.
23 challenger iterations rem

In [386]:
fdf.sort_values(by='result', ascending = False)

,challenger,control,result
AGAC,3730,3126,604
AGA,6203,5647,556
CTAT,5383,4845,538
CACA,3177,2644,533
GAAT,7663,7168,495
...,...,...,...
CCCA,4308,4882,-574
GCGG,1590,2255,-665
GCCA,3787,4460,-673
CG,844,1527,-683
